In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformer_flux_ca import FluxTransformer2DModelCA
# from diffusers import FluxTransformer2DModel
import torch

weight_dtype = torch.bfloat16
device = 'cuda'
# transformer = FluxTransformer2DModel.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=weight_dtype, subfolder='transformer')
# transformer = FluxTransformer2DModelCA.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=weight_dtype, subfolder='transformer', low_cpu_mem_usage=False, device_map=None)
transformer = FluxTransformer2DModelCA.from_pretrained("black-forest-labs/FLUX.1-Krea-dev", torch_dtype=weight_dtype, subfolder='transformer', low_cpu_mem_usage=False, device_map=None)

/home/cvlab17/anaconda3/envs/faceswap_omini/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Please 'pip install apex'


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

/home/cvlab17/anaconda3/envs/faceswap_omini/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cvlab17/anaconda3/envs/faceswap_omini/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: ./models/antelopev2/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExe

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of FluxTransformer2DModelCA were not initialized from the model checkpoint at black-forest-labs/FLUX.1-Krea-dev and are newly initialized: ['pulid_ca.19.norm1.weight', 'clip_vision_model.blocks.1.mlp.ffn_ln.bias', 'clip_vision_model.blocks.1.mlp.w1.weight', 'clip_vision_model.blocks.15.attn.k_proj.weight', 'clip_vision_model.blocks.10.norm2.bias', 'clip_vision_model.blocks.5.mlp.w3.weight', 'clip_vision_model.blocks.14.mlp.w2.weight', 'clip_vision_model.blocks.4.attn.rope.freqs_sin', 'pulid_ca.12.to_kv.weight', 'pulid_encoder.layers.7.0.to_kv.weight', 'clip_vision_model.blocks.19.attn.rope.freqs_cos', 'clip_vision_model.blocks.23.attn.proj.weight', 'clip_vision_model.blocks.21.mlp.ffn_ln.weight', 'clip_vision_model.blocks.18.norm2.weight', 'clip_vision_model.blocks.7.attn.inner_attn_ln.weight', 'pulid_encoder.layers.0.0.to_out.weight', 'pulid_ca.1.norm2.weight', 'clip_vision_model.blocks.6.norm2.weight', 'clip_vision_model.blocks.11.norm2.weight', 'pulid_encoder.mapping_2.

In [2]:
from pulid.utils import resize_numpy_image_long
from flux.sampling import denoise, get_noise, get_schedule, prepare, unpack, prepare_img, prepare_txt
from diffusers import FluxPipeline
from pipeline_flux_ca import FluxPipelineCA

device = 'cuda'
flux = FluxPipelineCA.from_pretrained("black-forest-labs/FLUX.1-Krea-dev", transformer=transformer, torch_dtype=weight_dtype).to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
flux.transformer.components_to_device(device)


import cv2
from glob import glob
from tqdm import tqdm
from PIL import Image

output_dir = './results_krea_0.9.1/vgg_src'
os.makedirs(output_dir, exist_ok=True)

src_img_path_list = sorted(glob('/mnt/data2/dataset/vgg_data/src/*.jpg'))[:1]
for src_img_path in tqdm(src_img_path_list, desc='Processing Image'):
    prompt="a photo of human face",
    neg_prompt = ""
    true_cfg = 1.0
    use_true_cfg = True if true_cfg > 1.0 else False

    src_num = os.path.basename(src_img_path).split('.')[0]
    id_image = cv2.imread(src_img_path)
    print(f"id_image shape: {id_image.shape}")
    id_image = cv2.cvtColor(id_image, cv2.COLOR_BGR2RGB)
    id_image = resize_numpy_image_long(id_image, 1024)
    id_image_pil = Image.fromarray(id_image)
    id_image_pil.save(f"{output_dir}/{src_num}_id.png")


    # flux.t5, flux.clip = flux.t5.to(device), flux.clip.to(device)
    # flux.pulid_model.components_to_device(device)
    id_embeddings, uncond_id_embeddings = flux.transformer.get_id_embedding(id_image, cal_uncond=use_true_cfg)
    id_embeddings = id_embeddings.to(device, dtype=weight_dtype)
    if use_true_cfg:
        uncond_id_embeddings = uncond_id_embeddings.to(device, dtype=weight_dtype)

    # Nan check
    if torch.isnan(id_embeddings).any():
        raise RuntimeError('id embedding is nan')

    # inp = prepare_txt(t5=flux.t5, clip=flux.clip, prompt=prompt, device=device)
    # inp_neg = prepare_txt(t5=flux.t5, clip=flux.clip,  prompt=neg_prompt, device=device) if use_true_cfg else None
    # flux.t5, flux.clip = flux.t5.cpu(), flux.clip.cpu()

    from diffusers.utils import make_image_grid
    # generate image
    # prompt = "a photo of human face"
    # negative_prompt = ""

    # prompt = "photo of a woman in red dress in a garden"
    # negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality, blurry"
    prompt = 'a photo of human face'
    negative_prompt = ''

    image_list = []

    g_list = [1.0, 2.0, 3.0, 4.0, 5.0]
    seed_list = [0, 1, 2, 3, 4]
    for g in g_list:
        for seed in seed_list:
            img = flux(
                prompt=prompt,
                negative_prompt=negative_prompt,
                width=512,
                height=512,
                num_inference_steps=30,
                guidance_scale=g,
                generator=torch.Generator('cpu').manual_seed(seed),
                id_embed=id_embeddings,
                uncond_id_embed=uncond_id_embeddings,
            )
            image = img.images[0] if isinstance(img.images, list) else img.images
            # image.save(f"{output_dir}/{src_num}.png")
            image_list.append(image)

    grid = make_image_grid(image_list, rows=len(g_list), cols=len(image_list)//len(g_list), resize=512)
    grid.save(f"{output_dir}/{src_num}.png")



    # g_list = [1.0, 2.0, 3.0, 4.0, 5.0]
    # for g in g_list:
    #     for seed in [0, 1, 2, 3, 4]:
    #         img = flux.generate_image_embed(
    #             width=512,
    #             height=512,
    #             num_steps=30,
    #             start_step=0,
    #             guidance=g,
    #             seed=seed,
    #             id_embed=id_embeddings,
    #             uncond_id_embed=uncond_id_embeddings,
    #             inp=inp,
    #             inp_neg=inp_neg,
    #             prompt=prompt,
    #             neg_prompt="",
    #             true_cfg=true_cfg,
    #             timestep_to_start_cfg=1,
    #             max_sequence_length=128,
    #         )[0]
    #         image = img[0] if isinstance(img, list) else img
    #         image_list.append(image)

    # grid = make_image_grid(image_list, rows=len(g_list), cols=len(image_list)//len(g_list), resize=512)
    # grid.save(f"{output_dir}/{src_num}.png")

Processing Image:   0%|          | 0/1 [00:00<?, ?it/s]

id_image shape: (512, 512, 3)
fail to detect face using insightface, extract embedding on align face


  0%|          | 0/30 [00:00<?, ?it/s]

Processing Image:   0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

: 